<a href="https://colab.research.google.com/github/dadang6842/AI-study/blob/main/assignments/LSTM_time_series_modeling_250802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://velog.io/@lazy_learner/LSTM-%EC%8B%9C%EA%B3%84%EC%97%B4-%EC%98%88%EC%B8%A1-%EB%AA%A8%EB%93%88-%EB%A7%8C%EB%93%A4%EA%B8%B0-1
- 위 블로그 참고
- 모델만 빌드

### 전체 과정 요약
1. reshape_dataset
- shape 변경(차원의 변화가 없긴 함), numpy 배열로 변경
2. split_sequence
- seq_x, seq_y(레이블) 생성
- single_output=True면 레이블이 하나, False면 여러 개
3. split_train_valid_dataset
- reshape_dataset, split_sequence 메서드 실행 수 validation data를 나눔
4. build_and_compile_lstm_model
- 다층 LSTM, 단층 LSTM일 때 / single_output=True, False일 때 모델 구조를 다르게 빌드
- return_sequence를 다르게 설정
- ModelCheckpoint, ReduceLROnPlateau, EarlyStopping 등의 콜백 사용하여 성능 개선
5. forecast_validation_dataset
- 검증 데이터셋 예측
6. calculate_metrics
- 성능 평가

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [10]:
class ForecastLSTM:
    def __init__(self, random_seed: int = 1234):
        self.random_seed = random_seed

In [11]:
# reshape input dataset
def reshape_dataset(self, df: pd.DataFrame) -> np.array:
    # y 컬럼을 데이터프레임의 맨 마지막 위치로 이동
    if "y" in df.columns:
        df = df.drop(columns=["y"]).assign(y=df["y"]) # assign(): Dataframe에 새 열을 할당
    else:
        raise KeyError("Not found target column 'y' in dataset.")

    # shape 변경 & numpy array로 변경(sequential dataset 생성을 위해)
    dataset = df.values.reshape(df.shape)
    return dataset

ForecastLSTM.reshape_dataset = reshape_dataset # ForecastLSTM 클래스에 reshape_dataset를 동적으로 붙임(monkey-patching)

sequential dataset
- single_output=True: seq_y가 한 개인 경우 -> 입력 시퀀스의 끝 인덱스부터 steps만큼 떨어진 값 한 개를 가져옴
- single_output=False: seq_y가 여러 개인 경우 -> 입력 시퀀스의 끝 인덱스부터 steps만큼의 값을 가져옴
- seq_x = dataset[i:idx_in, :-1] -> 행, 열 선택, 열을 마지막에서 한 칸 전까지 선택(마지막 컬럼은 타깃 y)
- seq_y = dataset[idx_out - 1 : idx_out, -1] -> 마지막 열 선택(y)


In [12]:
def split_sequences(
    self, dataset: np.array, seq_len: int, steps: int, single_output: bool
) -> tuple:

    # feature와 y 각각 sequential dataset을 반환할 리스트 생성
    X, y = list(), list()
    # sequence length와 step에 따라 sequential dataset 생성
    for i, _ in enumerate(dataset):
        # 입력 시퀀스의 끝 인덱스
        idx_in = i + seq_len
        # 출력 시퀀스의 끝 인덱스
        idx_out = idx_in + steps

        # 남은 데이터가 부족하면 반복 종료
        if idx_out > len(dataset):
            break

        seq_x = dataset[i:idx_in, :-1]
        if single_output:
            seq_y = dataset[idx_out - 1 : idx_out, -1]
        else:
            seq_y = dataset[idx_in:idx_out, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


ForecastLSTM.split_sequences = split_sequences

In [13]:
# split dataset
def split_train_valid_dataset(
    self,
    df: pd.DataFrame,
    seq_len: int,
    steps: int,
    single_output: bool,
    validation_split: float = 0.3,
    verbose: bool = True, # 데이터 분할 결과를 확인하고 싶을 때 True
) -> tuple:
    # dataframe을 numpy array로 reshape
    dataset = self.reshape_dataset(df=df)

    # feature와 y를 sequential dataset으로 분리
    X, y = self.split_sequences(
        dataset=dataset,
        seq_len=seq_len,
        steps=steps,
        single_output=single_output,
    )

    # X, y에서 validation dataset 분리
    dataset_size = len(X)
    train_size = int(dataset_size * (1 - validation_split))
    X_train, y_train = X[:train_size, :], y[:train_size, :]
    X_val, y_val = X[train_size:, :], y[train_size:, :]
    if verbose:
        print(f" >>> X_train: {X_train.shape}")
        print(f" >>> y_train: {y_train.shape}")
        print(f" >>> X_val: {X_val.shape}")
        print(f" >>> y_val: {y_val.shape}")
    return X_train, y_train, X_val, y_val


ForecastLSTM.split_train_valid_dataset = split_train_valid_dataset

One-Step Forecast(single_output=True)
- 한 시점만 예측
- 최종 출력이 스칼라여야 하므로 Dense(1)
- LSTM도 마지막 시점만 내보내도록 return_sequences=False

Multi-Step Forecast
- 다중 시점 예측
- 모델 최종 출력이 길이 N 벡터여야 하므로 Dense(steps)
- Flatten()을 추가하는 이유:
- return_sequences=True면 (batch_size, time_step, unit(은닉 상태의 차원))를 출력
- Dense layer는 2D 입력 (batch_size, features)를 기대, 따라서 (batch_size, time_step x unit)으로 2차원으로 펴 줌


In [14]:
# build LSTM
def build_and_compile_lstm_model(
    self,
    seq_len: int,
    n_features: int,
    lstm_units: list,
    learning_rate: float,
    dropout: float,
    steps: int,
    metrics: str,
    single_output: bool,
    last_lstm_return_sequences: bool = False,
    dense_units: list = None,
    activation: str = None,
):
    """
    LSTM 네트워크를 생성한 결과를 반환한다.

    :param seq_len: Length of sequences. (Look back window size)
    :param n_features: Number of features. It requires for model input shape.
    :param lstm_units: Number of cells each LSTM layers.
    :param learning_rate: Learning rate.
    :param dropout: Dropout rate.
    :param steps: Length to predict.
    :param metrics: Model loss function metric.
    :param single_output: Whether 'yhat' is a multiple value or a single value.
    :param last_lstm_return_sequences: Last LSTM's `return_sequences`. Allow when `single_output=False` only.
    :param dense_units: Number of cells each Dense layers. It adds after LSTM layers.
    :param activation: Activation function of Layers.
    """
    tf.random.set_seed(self.random_seed) # 시드를 고정함으로써 실험을 재현할 수 있음
    model = Sequential()

    if len(lstm_units) > 1:
        # 다층 LSTM
        # 첫 번째 LSTM layer
        model.add(
            LSTM(
                units=lstm_units[0],
                activation=activation,
                return_sequences=True, # 이후 레이어에 타임 스텝 별 은닉 상태 전체를 넘김
                input_shape=(seq_len, n_features),
            )
        )
        # 나머지 LSTM layer
        lstm_layers = lstm_units[1:]
        for i, n_units in enumerate(lstm_layers, start=1):
            # 마지막 LSTM layer
            if i == len(lstm_layers):
                if single_output:
                    return_sequences = False # 마지막 시점 은닉 상태 하나만
                else:
                    return_sequences = last_lstm_return_sequences # 파라미터에 따라
                model.add(
                    LSTM(
                        units=n_units,
                        activation=activation,
                        return_sequences=return_sequences,
                    )
                )
            else:
                model.add(
                    LSTM(
                        units=n_units,
                        activation=activation,
                        return_sequences=True,
                    )
                )
    else:
        # 단일 LSTM
        if single_output:
            return_sequences = False
        else:
            return_sequences = last_lstm_return_sequences
        model.add(
            LSTM(
                units=lstm_units[0],
                activation=activation,
                return_sequences=return_sequences,
                input_shape=(seq_len, n_features),
            )
        )

    if single_output: # 한 시점만 예측
        if dense_units:
            for n_units in dense_units:
                model.add(Dense(units=n_units, activation=activation))
        if dropout > 0:
            model.add(Dropout(rate=dropout))
        model.add(Dense(1)) # 출력: 스칼라 값 하나
    else:  # Multiple Output Step
        if last_lstm_return_sequences:
            model.add(Flatten())
        if dense_units:
            for n_units in dense_units:
                model.add(Dense(units=n_units, activation=activation))
        if dropout > 0:
            model.add(Dropout(rate=dropout))
        model.add(Dense(units=steps)) # steps개 값을 반환

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=MSE, metrics=metrics)
    return model


ForecastLSTM.build_and_compile_lstm_model = build_and_compile_lstm_model

ModelCheckpoint
-  Keras 모델을 학습하는 동안 일정한 간격으로 모델의 가중치를 저장하고, 최상의 성능을 보인 모델을 선택하는 기능을 제공
- 모델을 재학습할 때는 이전에 저장된 가중치를 불러와서 학습을 시작하면 됨 (load_weights)

ReduceLROnPlateau
- 검증 손실(validation loss)이 더 이상 개선되지 않을 때 학습률을 동적으로 감소시켜 모델의 학습을 돕는 기법
- 학습률을 낮춤으로써 파라미터 업데이트 크기를 줄여 최솟값 근처를 정교하게 탐색
- factor: learning rate를 감소시킬 비율
- patience: 손실값이 개선되지 않은 상태를 얼마나 허용할 것인지를 설정하는 정수값(얼마의 epoch 동안 기다릴 건지)

EarlyStopping(patience=patience)
- patience 에포크 동안 모니터링 지표가 개선되지 않으면 훈련을 조기 종료
- 디폴트: monitor="val_loss", mode="auto"


In [15]:
# model training
def fit_lstm(
    self,
    df: pd.DataFrame,
    steps: int,
    lstm_units: list,
    activation: str,
    dropout: float = 0,
    seq_len: int = 16,
    single_output: bool = False,
    epochs: int = 200,
    batch_size: int = None,
    steps_per_epoch: int = None,
    learning_rate: float = 0.001,
    patience: int = 10,
    validation_split: float = 0.3,
    last_lstm_return_sequences: bool = False,
    dense_units: list = None,
    metrics: str = "mse",
    check_point_path: str = None,
    verbose: bool = False,
    plot: bool = True,
):
    """
    LSTM 기반 모델 훈련을 진행한다.

    :param df: DataFrame for model train.
    :param steps: Length to predict.
    :param lstm_units: LSTM, Dense Layers
    :param activation: Activation function for LSTM, Dense Layers.
    :param dropout: Dropout ratio between Layers.
    :param seq_len: Length of sequences. (Look back window size)
    :param single_output: Select whether 'y' is a continuous value or a single value.
    """

    np.random.seed(self.random_seed)
    tf.random.set_seed(self.random_seed)

    # 훈련, 검증 데이터셋 생성
    (
        self.X_train,
        self.y_train,
        self.X_val,
        self.y_val,
    ) = self.split_train_valid_dataset(
        df=df,
        seq_len=seq_len,
        steps=steps,
        validation_split=validation_split,
        single_output=single_output,
        verbose=verbose,
    )

    # LSTM 모델 생성
    n_features = df.shape[1] - 1 # 라벨 제외
    self.model = self.build_and_compile_lstm_model(
        seq_len=seq_len,
        n_features=n_features,
        lstm_units=lstm_units,
        activation=activation,
        learning_rate=learning_rate,
        dropout=dropout,
        steps=steps,
        last_lstm_return_sequences=last_lstm_return_sequences,
        dense_units=dense_units,
        metrics=metrics,
        single_output=single_output,
    )

    # 콜백 함수 정의
    if check_point_path is not None:
        # create checkpoint
        checkpoint_path = f"checkpoint/lstm_{check_point_path}.h5"
        # 모델 적합 과정에서 best model 저장
        checkpoint = ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=False, # 전체 모델 구조 + 가중치를 함께 저장 (True면 가중치만 저장)
            save_best_only=True, # 지정한 모니터링 지표(monitor)가 개선되었을 때만 새 파일로 덮어씀
            monitor="val_loss",
            verbose=verbose,
        )
        rlr = ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=patience, verbose=verbose
        )
        callbacks = [checkpoint, EarlyStopping(patience=patience), rlr]
    else:
        rlr = ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=patience, verbose=verbose
        )
        callbacks = [EarlyStopping(patience=patience), rlr]

    # 모델 훈련
    self.history = self.model.fit(
        self.X_train,
        self.y_train,
        batch_size=batch_size,
        steps_per_epoch=steps_per_epoch, # 한 에포크(epoch)가 몇 스텝인지 (numpy 배열을 넘길 때는 내부 계산, 제너레이터를 쓸 경우 명시적 지정)
        validation_data=(self.X_val, self.y_val),
        epochs=epochs,
        use_multiprocessing=True, # 데이터 로딩을 다중 프로세스로 병렬 수행
        workers=8, # 몇 개의 워커 프로세스를 띄울지
        verbose=verbose,
        callbacks=callbacks, # 훈련 과정 중 특정 이벤트(에포크 종료, 성능 개선 등)가 발생할 때 추가 작업을 실행하는 함수들의 리스트
        shuffle=False, # 에포크 시작 전 데이터를 섞을지 여부, 시계열 데이터는 False로 두어 순서를 유지
    )

    # 훈련 종료 후 best model 로드
    if check_point_path is not None:
        self.model.load_weights(f"checkpoint/lstm_{check_point_path}.h5")

    # 모델링 과정 시각화
    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(self.history.history[f"{metrics}"])
        plt.plot(self.history.history[f"val_{metrics}"])
        plt.title("Performance Metric")
        plt.xlabel("Epoch")
        plt.ylabel(f"{metrics}")
        if metrics == "mape": # MAPE: 평균 절대 백분율 오차
            plt.axhline(y=10, xmin=0, xmax=1, color="grey", ls="--", alpha=0.5) # 수평 기준선을 그림 (예측 오차가 10% 이하인지)
        plt.legend(["Train", "Validation"], loc="upper right")
        plt.show()


ForecastLSTM.fit_lstm = fit_lstm

np.expand_dims(x_val, axis=0)
- X_val -> (seq_len, n_features)
- predict 메서드는 (batch_size, seq_len, n_features) 형태를 기대
- expand_dims로 맨 앞에(axis=0) 차원을 하나 추가

self.model.predict(x_val)[0]
- predict(x_val) 은 (1, steps) 또는 (1, …) 형태의 배열을 반환
- [0] 으로 첫(유일한) 배치의 결과만 꺼내어 1차원 배열로 만듦

In [16]:
def forecast_validation_dataset(self) -> pd.DataFrame:
    # 검증 데이터셋의 실제 값(y)과, 예측 값(yhat)을 저장할 리스트 생성
    y_pred_list, y_val_list = list(), list()

    # 훈련된 모델로 validation dataset에 대한 예측값 생성
    for x_val, y_val in zip(self.X_val, self.y_val): # zip(): X_val과 y_val을 쌍으로 묶어줌
        x_val = np.expand_dims(
            x_val, axis=0
        )  # (seq_len, n_features) -> (1, seq_len, n_features)
        y_pred = self.model.predict(x_val)[0]
        y_pred_list.extend(y_pred.tolist()) # append() 쓰면 2차원 배열 구조가 됨 (x)
        y_val_list.extend(y_val.tolist())
    return pd.DataFrame({"y": y_val_list, "yhat": y_pred_list}) # 실제값, 예측값 리스트로 dataframe 생


ForecastLSTM.forecast_validation_dataset = forecast_validation_dataset

In [17]:
# performance metric
def calculate_metrics(df_fcst: pd.DataFrame) -> dict:
    true = df_fcst["y"]
    pred = df_fcst["yhat"]

    mae = (true - pred).abs().mean()
    mape = (true - pred).abs().div(true).mean() * 100
    mse = ((true - pred) ** 2).mean()
    return {
        "mae": mae,
        "mape": mape,
        "mse": mse,
    }